In [1]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import string

# Download NLTK data
nltk.download('punkt')
nltk.download('stopwords')

# Step 1: Read the CSV file
df = pd.read_csv('linkedin_jobs.csv')  # Replace with your file name
text_data = df['job_description']  # Replace with the column name containing job descriptions
text_data

[nltk_data] Downloading package punkt to /Users/yufeiquan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/yufeiquan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


0       This position is based at our Luxembourg headq...
1       Background\nDiversity, Equity and Inclusion ar...
2       Job Title: Planview Administrator / Analyst\nL...
3       Location – Fountain Valley, CA\nPurpose\nHAEA ...
4       Business Process Analyst I\nPURPOSE\nAnalyze, ...
                              ...                        
2778    Basic Qualifications\n-  Experience in process...
2779    We are seeking a passionate and experienced Wo...
2780    We are seeking a skilled and technically profi...
2781    - Job Title: Business Professional - Marketing...
2782    An innovative technology company is hiring for...
Name: job_description, Length: 2783, dtype: object

In [3]:
skill_variations = {
    # Programming and Tools
    "python": ["python", "py"],
    "r": ["r", "r programming"],
    "sql": ["sql", "sql server", "nosql", "mysql", "postgresql", "oracle", "sql databases", "structured query language"],
    "vba": ["vba", "visual basic for applications"],
    "matlab": ["matlab", "matrix laboratory"],
    "programming": ["programming", "coding", "software development", "application development"],
    "java": ["java", "j2ee"],
    "javascript": ["javascript", "js", "node.js", "react.js", "angular.js", "vue.js"],
    "c++": ["c++", "cpp"],
    "c#": ["c#", "c sharp", ".net", "dot net"],
    "excel": ["excel", "advanced excel", "google sheets", "spreadsheet analysis"],
    "data visualization": ["data visualization", "visualization", "tableau", "power bi", "matplotlib", "seaborn", "ggplot", "dash", "plotly"],
    "etl": ["etl", "data pipelines", "data integration", "extract transform load"],
    "google big query": ["google big query", "big query"],

    # Cloud Platforms and Computing
    "cloud computing": ["cloud computing", "cloud", "aws", "azure", "google cloud", "gcp"],
    "aws": ["aws", "amazon web services"],
    "azure": ["azure", "microsoft azure"],
    "google cloud": ["google cloud", "gcp", "google cloud platform"],
    "adf": ["adf", "azure data factory"],
    "databricks": ["databricks"],
    "hadoop": ["hadoop", "apache hadoop"],
    "spark": ["spark", "apache spark"],
    "snowflake": ["snowflake"],
    "redshift": ["redshift"],
    "big data": ["big data", "data lakes"],

    # Machine Learning and AI
    "machine learning": ["machine learning", "ml", "ml algorithms", "automated learning"],
    "tensorflow": ["tensorflow", "tf"],
    "pytorch": ["pytorch", "torch"],
    "scikit-learn": ["scikit-learn", "sklearn", "scikit learn"],
    "deep learning": ["deep learning", "neural networks", "dl"],
    "artificial intelligence": ["artificial intelligence", "ai", "ai systems"],
    "nlp": ["natural language processing", "nlp", "text mining", "text analytics", "language models"],
    "mlops": ["mlops", "machine learning operations", "model deployment", "model monitoring"],

    # Analytics and Reporting
    "data analysis": ["data analysis", "data-driven decision making", "collecting and cleaning datasets"],
    "financial analysis": ["financial analysis", "financial forecasting", "business analysis"],
    "statistical analysis": ["statistical analysis", "statistics", "statistical modeling"],
    "business analysis": ["business analysis", "business requirements analysis"],
    "pricing analysis": ["pricing analysis", "pricing models", "dynamic pricing", "margin optimization"],
    "data modeling": ["data modeling", "semantic layer"],
    "ad hoc analysis": ["ad hoc analysis"],
    "dashboards": ["dashboards", "dashboard development"],
    "revenue management": ["revenue management", "pricing domains", "customer lifetime value analysis"],
    "optimization": ["optimization", "margin optimization", "cost-saving initiatives"],

    # Responsibilities (Actionable Tasks)
    "collaboration": [
        "collaborating with Data Architecture, Governance, and Business Teams",
        "acting as a liaison between stakeholders and technical teams",
        "partnering with internal teams, including sales and operations",
        "facilitating cross-functional collaboration to identify trends and challenges"
    ],
    "data governance": [
        "designing and implementing access controls",
        "ensuring data accuracy and validation",
        "establishing best practices for data governance and compliance",
        "monitoring data security standards"
    ],
    "training and enablement": [
        "developing training materials",
        "promoting self-service capabilities",
        "building data literacy resources",
        "delivering tailored training sessions"
    ],
    "reporting": [
        "creating financial reports",
        "designing dashboards",
        "maintaining and updating performance metrics",
        "tracking and analyzing operational data"
    ],
    "forecasting": [
        "building predictive models",
        "conducting revenue forecasting",
        "performing demand elasticity analysis",
        "evaluating trends for pricing strategies"
    ],
    "investment analysis": [
        "preparing capital investment data",
        "supporting asset valuation",
        "analyzing investment strategies",
        "evaluating liquidity"
    ],
    "inventory management": [
        "tracking and forecasting advertising and digital inventory",
        "optimizing yield",
        "forecasting inventory availability",
        "resolving campaign or programming conflicts"
    ],
    "project management": [
        "managing project workflows in Agile environments",
        "documenting user stories, epics, and technical requirements",
        "tracking tasks using JIRA",
        "ensuring alignment with strategic goals"
    ],
    "communication": [
        "presenting insights to stakeholders",
        "translating technical results for non-technical audiences",
        "preparing client-facing presentations",
        "influencing leadership through data-driven recommendations"
    ],

    # Tools and Technologies
    "sap": ["sap"],
    "operative.one": ["operative.one"],
    "freewheel": ["freewheel"],
    "google ad manager": ["google ad manager", "gam"],
    "nielsen dar": ["nielsen dar"],

    # Soft Skills and Learning
    "critical thinking": ["critical thinking", "analytical thinking"],
    "continuous learning": ["continuous learning", "self-learning", "lifelong learning"],
    "leadership": ["leadership", "mentoring", "team leadership"],
}

In [4]:
import pandas as pd
import nltk
import string

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer

# If not already done:
nltk.download('punkt')
nltk.download('stopwords')

# ------------------------------------------------------------------------------
# 1. Read the CSV file
# ------------------------------------------------------------------------------
df = pd.read_csv('linkedin_jobs.csv')  # Replace with your file name
df = df.dropna(subset=['job_description'])  # drop rows where job_description is NaN
text_data = df['job_description']

# ------------------------------------------------------------------------------
# 2. Preprocess the text
# ------------------------------------------------------------------------------
def preprocess_text(text):
    # Lowercase
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Tokenize
    tokens = word_tokenize(text)
    # Remove stopwords & keep only alphabetic tokens
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word.isalpha() and word not in stop_words]
    return tokens

df['Tokens'] = text_data.apply(preprocess_text)

# Create a clean text column (for TF-IDF vectorizer)
df['Cleaned_Text'] = df['Tokens'].apply(lambda x: ' '.join(x))

# ------------------------------------------------------------------------------
# 3. Extract n-grams (up to trigrams) using TF-IDF
# ------------------------------------------------------------------------------
vectorizer = TfidfVectorizer(
    ngram_range=(1, 3),    # capture unigrams, bigrams, trigrams
    stop_words='english',  # additional removal of common English words
    min_df=2,             # only consider n-grams that appear at least twice
    max_features=3000      # limit the features (tune based on your dataset size)
)
tfidf_matrix = vectorizer.fit_transform(df['Cleaned_Text'])
features = vectorizer.get_feature_names_out()

# Calculate average TF-IDF for each n-gram across all documents
avg_tfidf = tfidf_matrix.mean(axis=0).A1
feature_tfidf_pairs = list(zip(features, avg_tfidf))

# Sort by descending TF-IDF
feature_tfidf_pairs = sorted(feature_tfidf_pairs, key=lambda x: x[1], reverse=True)

print("Top 50 n-grams by TF-IDF score:")
for feature, score in feature_tfidf_pairs[:50]:
    print(f"{feature} : {score:.4f}")

# ------------------------------------------------------------------------------
# 4. Dictionary-based skill matching with synonyms/variations
#    (expand this dictionary as needed for your domain)
# ------------------------------------------------------------------------------

def extract_skills(token_list, skill_dict):
    """
    token_list: list of preprocessed tokens from one job description
    skill_dict: dict with key=canonical skill, value=list of synonyms/variations
    Returns a list of matched canonical skills found in the text.
    """
    found_skills = set()
    # Convert token_list to lower for matching (already lower, but just in case)
    tokens = [t.lower() for t in token_list]

    for skill_canonical, variations in skill_dict.items():
        for variant in variations:
            variant_tokens = variant.lower().split()
            if len(variant_tokens) == 1:
                # Single-word check
                if variant_tokens[0] in tokens:
                    found_skills.add(skill_canonical)
            else:
                # Multi-word check (e.g., "machine learning")
                # We'll look for consecutive matches in the token list
                for i in range(len(tokens) - len(variant_tokens) + 1):
                    if tokens[i:i+len(variant_tokens)] == variant_tokens:
                        found_skills.add(skill_canonical)
                        break

    return list(found_skills)

df['Key_Skills'] = df['Tokens'].apply(lambda x: extract_skills(x, skill_variations))

# ------------------------------------------------------------------------------
# 5. Inspect results
# ------------------------------------------------------------------------------
print("\nSample of extracted skills:")
print(df[['job_description', 'Key_Skills']].head(10))

[nltk_data] Downloading package punkt to /Users/yufeiquan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/yufeiquan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Top 50 n-grams by TF-IDF score:
data : 0.0621
experience : 0.0555
business : 0.0527
financial : 0.0401
work : 0.0398
management : 0.0362
skills : 0.0356
team : 0.0342
support : 0.0297
ability : 0.0296
analysis : 0.0291
including : 0.0261
research : 0.0255
requirements : 0.0254
systems : 0.0252
role : 0.0251
strong : 0.0246
analyst : 0.0245
risk : 0.0242
investment : 0.0239
job : 0.0238
position : 0.0237
company : 0.0237
required : 0.0230
benefits : 0.0228
information : 0.0227
development : 0.0224
years : 0.0215
process : 0.0210
operations : 0.0210
hr : 0.0209
project : 0.0206
knowledge : 0.0205
solutions : 0.0205
new : 0.0203
processes : 0.0202
teams : 0.0198
related : 0.0196
ensure : 0.0193
marketing : 0.0192
reporting : 0.0192
working : 0.0191
employee : 0.0190
employees : 0.0188
technical : 0.0185
environment : 0.0184
performance : 0.0184
status : 0.0181
client : 0.0181
product : 0.0178

Sample of extracted skills:
                                     job_description  \
0  This posi

In [5]:
from collections import Counter

# Flatten the list of skills and count occurrences
all_skills = [skill for skills in df['Key_Skills'] if isinstance(skills, list) for skill in skills]
skill_counts = Counter(all_skills)

# Convert to a DataFrame for better visualization
skills_df = pd.DataFrame(skill_counts.items(), columns=['Skill', 'Count']).sort_values(by='Count', ascending=False)

# Save or inspect the full DataFrame as needed
skills_df.to_csv('skill_counts.csv', index=False)  # Optional: Save to a file

In [6]:
skills_df

,Skill,Count
2,excel,1072
4,leadership,754
12,sql,671
8,data visualization,529
22,data analysis,529
1,programming,478
0,python,426
18,statistical analysis,388
5,financial analysis,363
26,dashboards,322
